# Extracting Human Faces From Images

## Imports

In [21]:
import cv2 
import numpy as np 
import os

from huggingface_hub import hf_hub_download
from PIL import Image
from supervision import Detections
from ultralytics.engine.results import Results
from ultralytics import YOLO

## Use Model

In [22]:
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
model = YOLO(model_path)

## Test

In [ ]:
image_path = "/home/pooh/Downloads/IMG_20250418_233052.jpg"
original_image_bgr = cv2.imread(image_path)

SyntaxError: unterminated string literal (detected at line 1) (2309021931.py, line 1)

In [ ]:
if original_image_bgr is None:
    print(f"Error: Could not load image from path: {image_path}")
    # Handle the error appropriately, e.g., exit or raise an exception
    exit() # Exit the script if image loading fails
else:
    print(f"Image loaded successfully: {image_path}")

    # --- Perform inference ---
    # It's often efficient to pass the loaded image array directly
    output: list[Results] = model(original_image_bgr) # Pass the BGR image array
    results_ultralytics: Results = output[0] # Get results for the first (only) image

    # --- Get bounding boxes ---
    # Access the bounding boxes (xyxy format) from the results
    # .cpu() moves tensor to CPU, .numpy() converts to NumPy array
    bboxes_xyxy = results_ultralytics.boxes.xyxy.cpu().numpy()

    # Define the directory to save cropped faces
    output_crop_dir = "cropped_faces"
    os.makedirs(output_crop_dir, exist_ok=True) # Create directory if it doesn't exist

    # Define a base name for the output files
    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    print(f"Found {len(bboxes_xyxy)} face(s). Cropping and saving...")

    # --- Iterate through detected faces and crop ---
    if len(bboxes_xyxy) > 0:
        for i, bbox in enumerate(bboxes_xyxy):
            # Extract integer coordinates (xmin, ymin, xmax, ymax)
            x_min, y_min, x_max, y_max = map(int, bbox[:4])

            # Add padding or ensure coordinates are within bounds if necessary (optional)
            # Example: Adding 10px padding, ensuring it stays within image limits
            padding = 0 # Set padding pixels if desired
            h, w = original_image_bgr.shape[:2]
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)

            # Crop the face region from the original BGR image using NumPy slicing
            cropped_face_bgr = original_image_bgr[y_min:y_max, x_min:x_max]

            # --- Save the cropped face ---
            # Check if the crop is valid (has width and height)
            if cropped_face_bgr.shape[0] > 0 and cropped_face_bgr.shape[1] > 0:
                # Convert the cropped BGR face to RGB for saving with PIL
                cropped_face_rgb = cv2.cvtColor(cropped_face_bgr, cv2.COLOR_BGR2RGB)

                # Convert the NumPy array (RGB) to a PIL Image object
                cropped_face_pil = Image.fromarray(cropped_face_rgb)

                # Define the output path for this specific cropped face
                output_crop_path = os.path.join(output_crop_dir, f"{base_filename}_face_{i}.jpg")

                # Save the cropped face image
                cropped_face_pil.save(output_crop_path)
                print(f"Saved cropped face {i} to: {output_crop_path}")
            else:
                print(f"Warning: Skipped saving face {i} due to invalid dimensions after cropping.")
    else:
        print("No faces were detected in the image.")

Image loaded successfully: /home/pooh/Downloads/ai-generated-professional-portrait-1170x686.jpeg

0: 384x640 1 FACE, 34.0ms
Speed: 1.3ms preprocess, 34.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Found 1 face(s). Cropping and saving...
Saved cropped face 0 to: cropped_faces/ai-generated-professional-portrait-1170x686_face_0.jpg
